In [14]:
pip install wget


In [105]:
import pandas as pd
import wget 
from zipfile import ZipFile
from google.colab.data_table import DataTable
import plotly.graph_objects as go

**Link do Site CVM**

In [16]:
url_base = 'http://dados.cvm.gov.br/dados/CIA_ABERTA/DOC/DFP/DADOS/'

**Armazenando os dados do link em uma lista**

In [33]:
arquivos_zip = []
for ano in range(2010,2021):
  arquivos_zip.append(f'dfp_cia_aberta_{ano}.zip')

arquivos_zip

['dfp_cia_aberta_2010.zip',
 'dfp_cia_aberta_2011.zip',
 'dfp_cia_aberta_2012.zip',
 'dfp_cia_aberta_2013.zip',
 'dfp_cia_aberta_2014.zip',
 'dfp_cia_aberta_2015.zip',
 'dfp_cia_aberta_2016.zip',
 'dfp_cia_aberta_2017.zip',
 'dfp_cia_aberta_2018.zip',
 'dfp_cia_aberta_2019.zip',
 'dfp_cia_aberta_2020.zip']

**Realizando o dowload dos dados**



In [35]:
for arqu in arquivos_zip:
  wget.download(url_base + arqu)

**Descompactando os arquivos na pasta CVM**



In [37]:
for arq in arquivos_zip:
  ZipFile(arqu, 'r').extractall('CVM')

**Salvando os arquivos em um unico DataFrame**

In [41]:
nomes = ['BPA_con', 'BPA_ind', 'BPP_con', 'BPP_ind', 'DFC_MD_ind', 'DFC_MI_con','DMPL_con', 'DMPL_ind', 'DRE_con', 'DRE_ind', 'DVA_con', 'DVA_ind']
for nome in nomes:
  arquivo = pd.DataFrame()
  for ano in range(2010,2021):
    arquivo = pd.concat([arquivo, pd.read_csv(f'CVM/dfp_cia_aberta_{nome}_{ano}.csv', sep=';', decimal=',', encoding='ISO-8859-1')])
  arquivo.to_csv(f'DADOS_DFP/dfp_cia_aberta_{nome}_2010-2020.csv', index=False)

In [55]:
dre = pd.read_csv('/content/DADOS_DFP/dfp_cia_aberta_DRE_con_2010-2020.csv')

In [56]:
dre = dre[dre['ORDEM_EXERC'] == 'ÚLTIMO']

In [57]:
dre.head()

,CNPJ_CIA,DT_REFER,VERSAO,DENOM_CIA,CD_CVM,GRUPO_DFP,MOEDA,ESCALA_MOEDA,ORDEM_EXERC,DT_INI_EXERC,DT_FIM_EXERC,CD_CONTA,DS_CONTA,VL_CONTA,ST_CONTA_FIXA
1,00.000.000/0001-91,2010-12-31,3,BCO BRASIL S.A.,1023,DF Consolidado - Demonstração do Resultado,REAL,MIL,ÚLTIMO,2010-01-01,2010-12-31,3.01,Receitas da Intermediação Financeira,85143206.0,S
3,00.000.000/0001-91,2010-12-31,3,BCO BRASIL S.A.,1023,DF Consolidado - Demonstração do Resultado,REAL,MIL,ÚLTIMO,2010-01-01,2010-12-31,3.01.01,Receita de Juros,85143206.0,S
5,00.000.000/0001-91,2010-12-31,3,BCO BRASIL S.A.,1023,DF Consolidado - Demonstração do Resultado,REAL,MIL,ÚLTIMO,2010-01-01,2010-12-31,3.02,Despesas da Intermediação Financeira,-50775741.0,S
7,00.000.000/0001-91,2010-12-31,3,BCO BRASIL S.A.,1023,DF Consolidado - Demonstração do Resultado,REAL,MIL,ÚLTIMO,2010-01-01,2010-12-31,3.02.01,Despesa de Juros,-43061187.0,S
9,00.000.000/0001-91,2010-12-31,3,BCO BRASIL S.A.,1023,DF Consolidado - Demonstração do Resultado,REAL,MIL,ÚLTIMO,2010-01-01,2010-12-31,3.02.02,Despesa com Provisão para Perdas em Empréstimos,-7714554.0,S


In [66]:
empresas = dre[['DENOM_CIA','CD_CVM']].drop_duplicates().set_index('CD_CVM')
DataTable(empresas)

,DENOM_CIA
CD_CVM,
1023,BCO BRASIL S.A.
14206,BRB BCO DE BRASILIA S.A.
2437,CENTRAIS ELET BRAS S.A. - ELETROBRAS
14451,CIA ENERGETICA DE BRASILIA
14540,VBC ENERGIA SA
...,...
24880,LM TRANSPORTES INTERESTADUAIS SERVIÇOS E COMÉR...
24929,TIM S.A.
25496,JALLES MACHADO S.A.


In [67]:
empresa = dre[dre['CD_CVM'] == 7617]

In [68]:
empresa.head()

,CNPJ_CIA,DT_REFER,VERSAO,DENOM_CIA,CD_CVM,GRUPO_DFP,MOEDA,ESCALA_MOEDA,ORDEM_EXERC,DT_INI_EXERC,DT_FIM_EXERC,CD_CONTA,DS_CONTA,VL_CONTA,ST_CONTA_FIXA
20638,61.532.644/0001-15,2010-12-31,1,ITAUSA S.A.,7617,DF Consolidado - Demonstração do Resultado,REAL,MIL,ÚLTIMO,2010-01-01,2010-12-31,3.01,Receita de Venda de Bens e/ou Serviços,5240000.0,S
20640,61.532.644/0001-15,2010-12-31,1,ITAUSA S.A.,7617,DF Consolidado - Demonstração do Resultado,REAL,MIL,ÚLTIMO,2010-01-01,2010-12-31,3.02,Custo dos Bens e/ou Serviços Vendidos,-3624000.0,S
20642,61.532.644/0001-15,2010-12-31,1,ITAUSA S.A.,7617,DF Consolidado - Demonstração do Resultado,REAL,MIL,ÚLTIMO,2010-01-01,2010-12-31,3.03,Resultado Bruto,1616000.0,S
20644,61.532.644/0001-15,2010-12-31,1,ITAUSA S.A.,7617,DF Consolidado - Demonstração do Resultado,REAL,MIL,ÚLTIMO,2010-01-01,2010-12-31,3.04,Despesas/Receitas Operacionais,-15830000.0,S
20646,61.532.644/0001-15,2010-12-31,1,ITAUSA S.A.,7617,DF Consolidado - Demonstração do Resultado,REAL,MIL,ÚLTIMO,2010-01-01,2010-12-31,3.04.01,Despesas com Vendas,0.0,S


In [71]:
DataTable(empresa[['CD_CONTA', 'DS_CONTA']].drop_duplicates().set_index('CD_CONTA'))

,DS_CONTA
CD_CONTA,
3.01,Receita de Venda de Bens e/ou Serviços
3.02,Custo dos Bens e/ou Serviços Vendidos
3.03,Resultado Bruto
3.04,Despesas/Receitas Operacionais
3.04.01,Despesas com Vendas
3.04.02,Despesas Gerais e Administrativas
3.04.03,Perdas pela Não Recuperabilidade de Ativos
3.04.04,Outras Receitas Operacionais
3.04.05,Outras Despesas Operacionais


In [72]:
conta = empresa[empresa['CD_CONTA'] == '3.99.02.02']

In [76]:
conta.index = pd.to_datetime(conta['DT_REFER'])
conta

,CNPJ_CIA,DT_REFER,VERSAO,DENOM_CIA,CD_CVM,GRUPO_DFP,MOEDA,ESCALA_MOEDA,ORDEM_EXERC,DT_INI_EXERC,DT_FIM_EXERC,CD_CONTA,DS_CONTA,VL_CONTA,ST_CONTA_FIXA
DT_REFER,,,,,,,,,,,,,,,
2010-12-31,61.532.644/0001-15,2010-12-31,1,ITAUSA S.A.,7617,DF Consolidado - Demonstração do Resultado,REAL,MIL,ÚLTIMO,2010-01-01,2010-12-31,3.99.02.02,PN,1.01000,N
2011-12-31,61.532.644/0001-15,2011-12-31,1,ITAUSA S.A.,7617,DF Consolidado - Demonstração do Resultado,REAL,MIL,ÚLTIMO,2011-01-01,2011-12-31,3.99.02.02,PN,1.10000,N
2012-12-31,61.532.644/0001-15,2012-12-31,1,ITAUSA S.A.,7617,DF Consolidado - Demonstração do Resultado,REAL,MIL,ÚLTIMO,2012-01-01,2012-12-31,3.99.02.02,PN,0.94000,N
2013-12-31,61.532.644/0001-15,2013-12-31,1,ITAUSA S.A.,7617,DF Consolidado - Demonstração do Resultado,REAL,MIL,ÚLTIMO,2013-01-01,2013-12-31,3.99.02.02,PN,1.05000,N
2014-12-31,61.532.644/0001-15,2014-12-31,1,ITAUSA S.A.,7617,DF Consolidado - Demonstração do Resultado,REAL,MIL,ÚLTIMO,2014-01-01,2014-12-31,3.99.02.02,PN,1.30000,N
2015-12-31,61.532.644/0001-15,2015-12-31,1,ITAUSA S.A.,7617,DF Consolidado - Demonstração do Resultado,REAL,MIL,ÚLTIMO,2015-01-01,2015-12-31,3.99.02.02,PN,1.31000,N
2016-12-31,61.532.644/0001-15,2016-12-31,1,ITAUSA S.A.,7617,DF Consolidado - Demonstração do Resultado,REAL,MIL,ÚLTIMO,2016-01-01,2016-12-31,3.99.02.02,PN,1.11000,N
2017-12-31,61.532.644/0001-15,2017-12-31,1,ITAUSA S.A.,7617,DF Consolidado - Demonstração do Resultado,REAL,MIL,ÚLTIMO,2017-01-01,2017-12-31,3.99.02.02,PN,1.13000,N
2018-12-31,61.532.644/0001-15,2018-12-31,1,ITAUSA S.A.,7617,DF Consolidado - Demonstração do Resultado,REAL,MIL,ÚLTIMO,2018-01-01,2018-12-31,3.99.02.02,PN,1.13000,N


### **Calculando o PL**

In [87]:
pip install yfinance

In [88]:
import yfinance as yf

**Pegando os valores da ação de fechamento**

In [89]:
prices = yf.download('ITSA4.SA', start='2011-01-01')[['Adj Close', 'Close']]

[*********************100%***********************]  1 of 1 completed


**Exibindo os valores**

In [94]:
prices.head()

,Adj Close,Close
Date,,
2011-01-03,4.355384,7.283394
2011-01-04,4.436381,7.377124
2011-01-05,4.492749,7.470855
2011-01-06,4.396596,7.310962
2011-01-07,4.267283,7.095933


**Pegando os valores das ações de fechamento e o valor da conta é o lucro por ação**

In [95]:
indicadores = prices.join(conta['VL_CONTA'], how='outer')

In [98]:
indicadores.rename({'VL_CONTA':'LPA'}, axis=1, inplace=True)
indicadores

,Adj Close,Close,LPA
2010-12-31,NaN,NaN,1.01
2011-01-03,4.355384,7.283394,NaN
2011-01-04,4.436381,7.377124,NaN
2011-01-05,4.492749,7.470855,NaN
2011-01-06,4.396596,7.310962,NaN
...,...,...,...
2021-03-08,9.944682,9.960000,NaN
2021-03-09,10.034544,10.050000,NaN
2021-03-10,10.310000,10.310000,NaN
2021-03-11,10.280000,10.280000,NaN


O lucro por ações aparece apenas no ultimo periodo. Para que copie também nos demais periodos utiliza-se o "Fillna".

In [99]:
indicadores.fillna(method='ffill', inplace=True)

In [100]:
indicadores

,Adj Close,Close,LPA
2010-12-31,NaN,NaN,1.01000
2011-01-03,4.355384,7.283394,1.01000
2011-01-04,4.436381,7.377124,1.01000
2011-01-05,4.492749,7.470855,1.01000
2011-01-06,4.396596,7.310962,1.01000
...,...,...,...
2021-03-08,9.944682,9.960000,0.83892
2021-03-09,10.034544,10.050000,0.83892
2021-03-10,10.310000,10.310000,0.83892
2021-03-11,10.280000,10.280000,0.83892


In [102]:
indicadores.dropna(inplace=True)
indicadores

,Adj Close,Close,LPA
2011-01-03,4.355384,7.283394,1.01000
2011-01-04,4.436381,7.377124,1.01000
2011-01-05,4.492749,7.470855,1.01000
2011-01-06,4.396596,7.310962,1.01000
2011-01-07,4.267283,7.095933,1.01000
...,...,...,...
2021-03-08,9.944682,9.960000,0.83892
2021-03-09,10.034544,10.050000,0.83892
2021-03-10,10.310000,10.310000,0.83892
2021-03-11,10.280000,10.280000,0.83892


In [103]:
indicadores['PL'] = indicadores['Close'] / indicadores['LPA']
indicadores['PL_ajustado'] = indicadores['Adj Close'] / indicadores['LPA']

In [104]:
indicadores

,Adj Close,Close,LPA,PL,PL_ajustado
2011-01-03,4.355384,7.283394,1.01000,7.211281,4.312261
2011-01-04,4.436381,7.377124,1.01000,7.304083,4.392457
2011-01-05,4.492749,7.470855,1.01000,7.396886,4.448267
2011-01-06,4.396596,7.310962,1.01000,7.238576,4.353065
2011-01-07,4.267283,7.095933,1.01000,7.025676,4.225033
...,...,...,...,...,...
2021-03-08,9.944682,9.960000,0.83892,11.872407,11.854148
2021-03-09,10.034544,10.050000,0.83892,11.979688,11.961264
2021-03-10,10.310000,10.310000,0.83892,12.289611,12.289611
2021-03-11,10.280000,10.280000,0.83892,12.253850,12.253850


In [106]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=indicadores.index, y=indicadores['PL'], name='PL'))
fig.add_trace(go.Scatter(x=indicadores.index, y=indicadores['PL_ajustado'], name='PL_ajustado'))
fig.add_trace(go.Scatter(x=indicadores.index, y=indicadores['Close'], name='ITASA4'))
fig.add_trace(go.Scatter(x=indicadores.index, y=indicadores['Adj Close'], name='ITASA4_ajustado'))